# Charge Density Calculator

This notebook contains code for calculating the charge densities resulting from candidate BLAES stimulation parameter sets. The estimated values are compared to the tissue damage thresholds reviewed in [Cogan et al. 2016](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5386002/). *Note that the returned values are approximations, as the actual charge will vary with tissue type, electrode material, etc*.

> Author: Justin Campbell  
> Contact: justin.campbell@hsc.utah.edu  
> Version: 02/01/24

---

#### **Charge density** ($\rho$) is defined as:
$$
\rho = \frac{Q}{A}
$$

$$
Q = I \times t  
$$
$$
A = 2\pi rh + \pi r^2
$$

where,  
<br>
$Q = \text{charge per phase (C)}$,  
$I = \text{current (A)}$,  
$t = \text{pulse width (s)}$,  
$A = \text{electrode contact surface area (mm2)}$,  
$r = \text{electrode contact radius (mm)}$,  
$h = \text{electrode contact height (mm)}$

#### **Cogan et al. 2016 (Figure 3)**
Meta-analysis of tissue damage thresholds during various forms of therapeutic electrical stimulation.

![Cogan Fig 3.](Cogan2016Fig3.jpg)

### Implementation

In [1]:
import numpy as np

In [2]:
def calcChargeDensity(I, t, elec = None, d = None, h = None, returnVars = False):
    '''
    Calculate the charge density of the electrode
    
    Inputs:
    I: current (mA)
    t: pulse-width (uS)
    elec: type of electrode ('PMT', 'AdTech', or 'DIXI') [optional]
    d: diameter of the electrode (mm) [optional]
    h: height of the electrode (mm) [optional]
    Return: if True, return the rho and Q
    
    Outputs:
    rho: charge density (uC / mm^2) [log10 scale]
    Q: charge per phase (uC) [log10 scale]
    '''
    
    # Electrode geometries (in mm); per Supplementary Table 2 in Davis et al. 2021
    if elec == 'PMT':
        # PMT2102
        d = 0.8
        h = 2
    elif elec == 'AdTech':
        # RD**R-SP05X-000
        d = 0.86
        h = 2.29
    elif elec == 'BF':
        #BF08R-SP05X-000
        d = 1.28
        h = 1.57
    elif elec == 'DIXI':
        # Microdeep D08
        d = 0.8
        h = 2
    elif (d is not None) and (h is not None):
        elec = 'Custom'
        pass
    else:
        print("Invalid electrode type! Must specify 'PMT', 'AdTech', 'DIXI', or values for d and h.")
        return
    
    Q = ((I/1e3) * (t/1e6)) * 1e6 # charge per phase (uC); convert mA to A and uS to s
    A = ((2 * np.pi * (d/2) * h) + (np.pi * (d/2)**2)) / 100 # surface area of the electrode (cm^2)
    rho = Q / A # charge density (uC / cm^2)
    
    # Convert rho and Q to facilitate comparison with Cogan et al. 2016
    rho_log10 = np.log10(rho) # convert rho to log10(uC / cm^2) 
    Q_log10 = np.log10(Q) # convert Q to log10(uC)
    
    if returnVars:
        return rho, Q
    else:
        # Display
        print('Electrode: %s (%.2fmm x %.2fmm):' % (elec, d, h))
        print('    Amperage: %.2f (mA)' % I)
        print('    Pulse-width: %.2f (uS)' % t)
        print('    Charge per phase: %.2f, %.2f (uC, log10)' %(Q, Q_log10))
        print('    Charge density: %.2f, %.2f (uC/cm^2, log10)' %(rho, rho_log10))
        print('')

In [3]:
# Test different amperages (mA) and pulse-widths (uS) for BLAES Paramater Optimization
for I in [1, 2]:
    for t in [250, 500]:
        calcChargeDensity(I, t, elec = 'PMT')

Electrode: PMT (0.80mm x 2.00mm):
    Amperage: 1.00 (mA)
    Pulse-width: 250.00 (uS)
    Charge per phase: 0.25, -0.60 (uC, log10)
    Charge density: 4.52, 0.66 (uC/cm^2, log10)

Electrode: PMT (0.80mm x 2.00mm):
    Amperage: 1.00 (mA)
    Pulse-width: 500.00 (uS)
    Charge per phase: 0.50, -0.30 (uC, log10)
    Charge density: 9.04, 0.96 (uC/cm^2, log10)

Electrode: PMT (0.80mm x 2.00mm):
    Amperage: 2.00 (mA)
    Pulse-width: 250.00 (uS)
    Charge per phase: 0.50, -0.30 (uC, log10)
    Charge density: 9.04, 0.96 (uC/cm^2, log10)

Electrode: PMT (0.80mm x 2.00mm):
    Amperage: 2.00 (mA)
    Pulse-width: 500.00 (uS)
    Charge per phase: 1.00, 0.00 (uC, log10)
    Charge density: 18.09, 1.26 (uC/cm^2, log10)

